# Utils

In [134]:
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt


imgCol = cv2.imread('../Utils/LenaJPEG.jpg')

def plotSignal(signal=[], title='', xmin = 0, xmax = 0):
  if(xmax == 0):
    xmax = signal.size

  plt.title(title)
  plt.plot(signal)
  plt.xlim(xmin, xmax)
  plt.show() 

def show_Image(img=[], resize=-1, title='img'):
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    if 0 < resize < 2:
      img = cv2.resize(img, [int(width*resize), int(height*resize)])
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def get_Image(path='../Utils/LenaJPEG.jpg', resize=-1):
    toRet = cv2.imread(path, 0)
    if 0 < resize < 2:
      if len(toRet.shape) == 3:
          height, width, depth = toRet.shape
      else :
          height, width = toRet.shape
      toRet = cv2.resize(toRet, [int(width*resize), int(height*resize)])
    return toRet

colors = [
  [255, 0, 0],   # Rot
  [0, 255, 0],   # Grün
  [0, 0, 255],   # Blau
  [255, 255, 0], # Gelb
  [255, 0, 255], # Magenta
  [0, 255, 255], # Cyan
  [128, 0, 0],   # Dunkelrot
  [0, 128, 0],   # Dunkelgrün
  [0, 0, 128],   # Dunkelblau
  [128, 128, 0], # Dunkelgelb
  [128, 0, 128], # Dunkelmagenta
  [0, 128, 128], # Dunkelcyan
  [192, 192, 192], # Hellgrau
  #[128, 128, 128], # Grau
  #[255, 255, 255], # Weiß
  #[0, 0, 0],     # Schwarz
  [255, 128, 128], # Hellrot
  [128, 255, 128], # Hellgrün
  [128, 128, 255], # Hellblau
  [255, 255, 128], # Hellgelb
  [255, 128, 255], # Hellmagenta
  [128, 255, 255], # Hellcyan
  [255, 0, 128],   # Pink
  [128, 255, 0],   # Limette
  [0, 128, 255],   # Himmelblau
  [255, 128, 0],   # Orange
  [128, 0, 255],   # Lila
  [0, 255, 128],   # Türkis
  [255, 128, 64],  # Pfirsich
  [64, 255, 128],  # Limonengrün
  [128, 64, 255],  # Violett
]
   

# Praktikum 01 (Not necessary)

In [135]:
def printNumberFrom1To100():
    for i in range(1, 101):
        print(i)

def fibunacci(n):
    a = [0]*(n+4)
    a[0] = 1
    a[1] = 1
    print(0)
    print(1)
    print(1)
    for i in range(2,n+3):
        a[i] = a[i-1]+a[i-2]
        print(a[i])

# fibunacci(10)

# for y in range(height):
#     for x in range(width):
#         imgCol[y][x] = pow(imgCol[y][x]/255, 1/2) * 255



# cv2.imshow("img", imgCol)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


# Praktikum 02

In [136]:
def getHistogramm(img=[]):
    histogram=np.zeros(256)
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    for y in range(height):
        for x in range(width):
            if img.ndim == 3:
                i = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            else:
                i = img[y][x]

            histogram[int(i)] += 1

    return histogram

def getNormHistogramm(img=[], h=[]):
    if len(h)==0:
        if len(img.shape) == 3:
            height, width, depth = img.shape
        else :
            height, width = img.shape
        return getHistogramm(img) / (width*height)
    else:
        return h/np.sum(h)

def getCumulatedHistogramm(img=[], h=[]):
    H = np.zeros(256)
    if len(h)==0:
        h = getNormHistogramm(img)
    else:
        h = getNormHistogramm(h=h)
    for i in range(len(h)):
        for j in range(i): 
            H[i] += h[j]
    return H

def linearContrast(img=[], t0=0, t1=255):
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    toRet = np.zeros((height, width), dtype='uint8')
    for y in range(height):
        for x in range(width):
            if img.ndim == 3:
                a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            else:
                a = img[y][x]
            if a < t0:
                toRet[y][x] = 0
            if t0 <= a < t1:
                toRet[y][x] = 255/(t1-t0) * (a-t0)
            if a >= t1:
                toRet[y][x] = 255
    return toRet

def autoContrast(img, schwellwert):
    a1_low = getPercentage(img, schwellwert)
    a1_high = getPercentage(img, 1-schwellwert)
    return linearContrast(img, a1_low, a1_high)

def getPercentage(img, schwellwert):
    if schwellwert < 0:
        schwellwert = 0
    elif schwellwert > 1:
        schwellwert = 1
    H = getCumulatedHistogramm(img)
    toRet = 255
    for i in range(len(H)):
        if H[i] >= schwellwert:
            toRet = i
            break
    return toRet

def getMedianOfHistogramm(img=[], h=[]):
    if len(h) == 0:
        H = getCumulatedHistogramm(img)
    else:
        H = getNormHistogramm(h=h)
    for i in range(len(H)):
        if H[i] >= 0.5:
            return i
def getGrey(img=[]):
    return cv2.cvtColor(cv2.imread('../Utils/LenaJPEG.jpg'), cv2.COLOR_BGR2GRAY)    


def getBinary(img=[], schwellwert=125):
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    toRet = np.zeros((height, width))
    for y in range(height):
        for x in range(width):
            a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            if a < schwellwert:
                toRet[y][x] = 0
            else:
                toRet[y][x] = 1
    return toRet

def same_image(img=[]):
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    toRet = np.zeros((width, height), dtype='uint8')
    for y in range(height):
        for x in range(width):
            a = 0.2125 * img[y][x][0] + 0.7154 * img[y][x][1] + 0.072 * img[y][x][2]
            # a = img[y][x][0]
            toRet[y][x] = a
            
    return toRet

testing Praktikum 02

In [137]:
# # print(getMedianOfHistogramm(imgCol))
# median = getMedianOfHistogramm(imgCol)
# print(median)
# img = getBinary(imgCol, median)
# # getNormHistogramm(img)
# # auto_con = autoContrast(imgCol, 0.1)

# # plotSignal(getCumulatedHistogramm(imgCol))
# show_Image(img)
# # show_Image(imgCol)

# Praktikum 03

In [138]:
# Not working
def Otsu_Threshold_naiv(Histrogramm=[], img=[]):
    toRet = -1
    varMax = -1
    width, height, depth = img.shape
    size = width * height
    K = len(Histrogramm)
    for i in range(K-1):
        w_0 = np.sum(Histrogramm[0:i]) / size
        w_1 = np.sum(Histrogramm[i:K]) / size
        my_0 = np.mean(Histrogramm[0:i])
        my_1 = np.mean(Histrogramm[i:K])
        varBetween = w_0 * w_1 * (my_0-my_1)**2
        if varMax < varBetween:
            varMax = varBetween
            toRet = i
    return toRet

def weigthed_Histogram(h=[]):
    toRet = 0
    for i in range(len(h)):
        toRet += i * h[i]
    return toRet


def Otsu_Threshold_efficient(img=[], vorwissen=False, t=1):
    """
    calculate the treshold for an linear regression

    :param img: the used Image
    :type img: Arraylike
    :param vorwissen: if you know how many persentage of the picture will be black
    :type vorwissen: boolean
    :param t: the known persentage, if vorwissen is True
    :type t: float in range [0;1]
    :return: treshold for further computing
    :rtype: int
    """
    h = getHistogramm(img)
    toRet = -1
    varMax = -1
    c_0 = 0
    sum_0 = 0 
    width, height, depth = img.shape
    size = width * height
    if vorwissen:
        K = getPercentage(img, t)
    else:
        K = len(h)
    for i in range(K):
        c_0 = c_0 + h[i]
        c_1 = size - c_0
        sum_0 = sum_0 + i * h[i]
        my_0 = sum_0 / c_0
        my_1 = ( weigthed_Histogram(h) - sum_0) / c_1
        varBetween = c_0 * c_1 * (my_0-my_1)**2
        if varMax < varBetween:
            varMax = varBetween
            toRet = i
    return toRet

def getEroded(img=[], schwellwert=125):
    toRet = getBinary(img, schwellwert)
    kernel = np.array([[0, 1, 0],
                       [1, 1, 1],
                       [0, 1, 0]],
                      dtype=np.uint8)
        
    toRet = cv2.erode(toRet, kernel)
    toRet = cv2.erode(toRet, kernel)
    toRet = cv2.dilate(toRet, kernel)
    return toRet

def region_labeling(img=[]):
    toRet = np.array(img, dtype=np.uint8)
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    m = 2
    for y in range(height):
        for x in range(width):
            if img[y][x] == 1:
                toRet = floodfill(img, x, y, m)
                m+=1
    for em in range(m):
        print('region_label: m:',em,'count:',np.count_nonzero(toRet == em))
    return toRet

def floodfill(img=[], x=0, y=0, m=1):
    toRet = img
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    if(x == 0 and y == 0):
        print(height, width)
    S = []
    S.append((x, y))
    
    while len(S) > 0:
        (x, y) = S.pop()
        if((0<=x<width and 0<=y<height) and img[y][x] == 1):
            toRet[y][x] = m
            S.append((x+1, y))
            S.append((x, y+1))
            S.append((x, y-1))
            S.append((x-1, y))
    return toRet

def computeProperties(img):
    R = {}
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    for y in range(height):
        for x in range(width):
            m = img[y][x]
            if(m not in R):
                R[m] = {'flaeche': 0, 'bb': {'xmin': x, 'xmax': x, 'ymin': y, 'ymax': y}}
            if(m >= 2):
                R[m]['flaeche'] += 1
                R[m]['bb']['xmin'] = min(R[m]['bb']['xmin'], x)
                R[m]['bb']['xmax'] = max(R[m]['bb']['xmax'], x)
                R[m]['bb']['ymin'] = min(R[m]['bb']['ymin'], y)
                R[m]['bb']['ymax'] = max(R[m]['bb']['ymax'], y)

                R[m]['bb']['height'] = R[m]['bb']['ymax'] - R[m]['bb']['ymin']
                R[m]['bb']['width'] = R[m]['bb']['xmax'] - R[m]['bb']['xmin']

    return R

def test_Properties(img=[]):
    if len(img.shape) == 3:
        height, width, depth = img.shape
    else :
        height, width = img.shape
    toRet = np.zeros([height, width, 3], dtype=np.uint8)
    for y in range(height):
        for x in range(width):
            m = img[y][x]
            if(m>=2):
                toRet[y][x] = colors[m]
    return toRet

def draw_Bounding_Boxes(img, R):
    for m in range(2,8):
        for x in range(R[m]['bb']['xmin'], R[m]['bb']['xmax']):
            y0 = R[m]['bb']['ymax']
            img[y0][x] = colors[m+1]
            y1 = R[m]['bb']['ymin']
            img[y1][x] = colors[m+1]
        for y in range(R[m]['bb']['ymin'], R[m]['bb']['ymax']):
            x = R[m]['bb']['xmax']
            img[y][x] = colors[m+1]
            x = R[m]['bb']['xmin']
            img[y][x] = colors[m+1]
            
    return img

def draw_Contour(img=[], start=[], end=[]):

    toRet = img
    # Berechnen der Schritte zwischen Start und Ende
    delta_x = end[0] - start[0]
    delta_y = end[1] - start[1]

    # Bestimmen der Vorzeichen der Schritte
    sign_x = 1 if delta_x >= 0 else -1
    sign_y = 1 if delta_y >= 0 else -1

    # Iterieren über alle Pixel zwischen Start und Ende
    x, y = start
    while x != end[0] or y != end[1]:
        # if(len(img) == 2):
        toRet[x][y][0] = 255
        toRet[x][y][1] = 255
        toRet[x][y][2] = 255
        # else:
        #     img[x][y] = (255, 0, 0)
        x += sign_x
        y += sign_y

    return toRet



testing Praktikum 03

In [139]:
# SetExmpl = cv2.imread('../Utils/SetWellen.png')
SetExmpl = get_Image('../Utils/SetWellen.png', resize=0.3)

otsu_efficient = Otsu_Threshold_efficient(SetExmpl)
# ret, thresh = cv2.threshold(SetExmpl, thresh=cv2.THRESH_BINARY + cv2.THRESH_OTSU, maxval=255, type=cv2.THRESH_OTSU)
print(otsu_efficient)

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:

# binary = getEroded(SetExmpl, 130)
# label = region_labeling(binary)
# R = computeProperties(label)
# gray_image = linearContrast(label, 0, 255)
# tst = test_Properties(gray_image)
# bb = draw_Bounding_Boxes(tst, R)
# bb = draw_Contour(bb, [30, 30], [100, 100])
# show_Image(bb)

Contours_IMG = cv2.imread('../Utils/SetWellen.png', 0)
contours, hierarchy = cv2.findContours(Contours_IMG, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(Contours_IMG, contours, 1, [255,0,0])
show_Image(Contours_IMG)
# print(contours[1][0][0])
# print(contours[1][1][0])
# IMG = draw_Contour(Contours_IMG, contours[1][0][0], contours[1][1][0])
# show_Image(IMG)

In [ ]:
test_img = np.ones((255,255), dtype=np.uint8)
for y in range(255):
    for x in range(255):
        if(abs(x-y)<2):
            test_img[y][x] = 255
test_img = region_labeling(test_img)
show_Image(test_img)

region_label: m: 0 count: 0
region_label: m: 1 count: 0
region_label: m: 2 count: 32131
region_label: m: 3 count: 32131


# Praktikum 04

In [147]:
def edge_Sharpening(img=[], w=1):
    return np.uint8(cv2.Laplacian(img, -1, scale=w,))
    # return np.subtract(img, cv2.Laplacian(img, 0, scale=w))

def gaussian_Blur(img=[], sigmax=1, sigmay=1, alpha = 1):

    # return img - cv2.GaussianBlur(img, (5,5), sigmaX=sigmax, sigmaY=sigmay)
    return np.uint8((1 + alpha) * img - alpha * cv2.GaussianBlur(img, (7,7), sigmaX=sigmax, sigmaY=sigmay))

def Canny_edge_Detector(img=[], threshold1=1, threshold2=1):
    return cv2.Canny(img, threshold1, threshold2)

testing Prak 04

In [151]:
img = get_Image()
grey = getGrey(img)
# show_Image(grey)
sharpend_img = edge_Sharpening(grey, 1)
blurred_img = gaussian_Blur(img, 1, 1, .9)
Canny_img = Canny_edge_Detector(img, 2.5, 2.5)
show_Image(img)
# show_Image(sharpend_img)
show_Image(blurred_img)
# show_Image(Canny_img)